In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
location = r"/content/drive/My Drive/Files/MAML/"   

# Tasks:
1. Load Libraries ,   Load BERT_Tokenizer, BERT_model 
2. Define NLI model class and its trainer and prediction function
3. Write a function to perform NLI on test data 2,3 exmples
4. import mBERT 

**Dataset Details**
1. Context
2. Hypothesis
3. Enatilment
        "entailed" : 0
        "not-entailed" : 1
4. Topic_Label
        "india" : 0
        "news" : 1
        "international" : 2
        "entertainment" : 3
        "sport" : 4
        "science" : 5






### Libraries

In [3]:
!pip install transformers
!pip install torchmeta

In [4]:
import pandas as pd
import numpy as np

In [5]:
import torch
import torch.nn as nn

from torch import optim
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchmeta.utils.data import BatchMetaDataLoader
from torchmeta.utils.gradient_based import gradient_update_parameters

from torchmeta.modules import *

In [6]:
from collections import OrderedDict
import tqdm.notebook as tq
import random
from sklearn.model_selection import train_test_split

In [7]:
!git clone https://github.com/AI4Bharat/indic-bert

fatal: destination path 'indic-bert' already exists and is not an empty directory.


In [8]:
!pip install -r /content/indic-bert/requirements_colab.txt

In [9]:
from transformers import BertTokenizer, BertModel

from transformers import AutoTokenizer, AutoModel
  
BERT_Tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

BERT_Model = AutoModel.from_pretrained("ai4bharat/indic-bert")

#BERT_Tokenizer  = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
#BERT_Model = BertModel.from_pretrained("bert-base-multilingual-cased")

### Load File and Preprocessing
BERT Tokenizer and Dataloader Generation

In [10]:
def read_file(location, train, test):
    df_train = {}
    df_test = {}
    for lang in train:
        df_train[lang] = pd.read_csv(location + lang + '.csv')
    for lang in test:
        df_test[lang]  = pd.read_csv(location + lang + '.csv')
    return df_train, df_test

In [11]:
def process_pair( context, hypothesis, max_length ):
    tcontext = BERT_Tokenizer.encode(context, max_length=max_length,  truncation=True, add_special_tokens = False)
    thypothesis = BERT_Tokenizer.encode(hypothesis, max_length=max_length, truncation=True, add_special_tokens = False)
    len_c, len_h = len(tcontext), len(thypothesis)

    token_id =  [BERT_Tokenizer.cls_token_id
                ] + tcontext + [BERT_Tokenizer.sep_token_id
                ] + thypothesis + [BERT_Tokenizer.sep_token_id]
    segment_id = [0]*(len_c+2) + [1] *(len_h + 1)
    mask_id    = [1]*(len_c + len_h + 3)
    return  torch.tensor(token_id, device=device),  torch.tensor(segment_id, device=device), torch.tensor(mask_id, device=device)

def process_dataframe( df, max_length ):
    token_ids = []
    mask_ids = []
    segment_ids = []
    y = []

    contexts = df['Context'].to_list()
    hypothesis = df['Hypothesis'].to_list()
    entailment = df['Entailment_Label'].to_list()
    for context, hypothesis, ent in  zip(contexts, hypothesis, entailment):
        
        token_id, segment_id, mask_id = process_pair( context, hypothesis, max_length )

        token_ids.append(token_id)
        segment_ids.append(segment_id)
        mask_ids.append(mask_id)
        y.append( int(ent == 'entailed') )
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids  = pad_sequence(mask_ids, batch_first=True)
    segment_ids = pad_sequence(segment_ids, batch_first=True, padding_value=1)
    y         = torch.tensor(y, device=device)
    return TensorDataset(token_ids, mask_ids, segment_ids, y)

In [12]:
def get_batch(dataset, train_length, test_length, batch_size=50, max_length=56):
    batch = []
    for lang in train_langs:
        supp = process_dataframe(dataset[lang].sample(train_length), max_length)
        quer = process_dataframe(dataset[lang].sample(test_length), max_length)
        support_loader = DataLoader( supp, shuffle=True, batch_size=train_length)
        query_loader   = DataLoader( quer, shuffle=True, batch_size=test_length)
        batch.append( { 'support' : support_loader, 'query' : query_loader } )
    return batch

### NLI Neural Network Model and Trainer Functions

In [13]:
class BERT_for_NLI(MetaModule):
    def __init__(self, BERT_Model, wt):
        super(BERT_for_NLI, self).__init__()
        self.BERT_Model = BERT_Model
        for param in self.BERT_Model.encoder.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(0.6)
        self.dense1 = MetaLinear(768, 256)
        self.act1 = nn.LeakyReLU(0.2)
        self.dense2 = MetaLinear(256, 2)
        self.act2 = nn.Softmax(dim=1)
        torch.nn.init.normal_(self.dense1.weight, mean = 0, std = wt)
        torch.nn.init.normal_(self.dense2.weight, mean = 0, std = wt)                      

    def forward( self, input_ids , mask_ids ,  segment_ids , params = None ):
        _, bert_embed = self.BERT_Model(
            input_ids,
            attention_mask       = mask_ids,
            token_type_ids       = segment_ids
        )
        dropped = self.dropout(bert_embed)
        d1out = self.dense1(dropped, params=self.get_subdict(params, 'dense1'))
        z1out = self.act1(d1out)
        d2out = self.dense2(z1out, params=self.get_subdict(params, 'dense2'))
        z2out = self.act2(d2out)
        return z2out

In [14]:
def get_accuracy(pred, target):
    #return  round( torch.mean((pred>=0.5).squeeze().eq(target).float() ).item() , 2)
    return round( torch.mean( pred.data.topk(1)[1].eq(target).float() ).item() , 2)

In [15]:
def meta_train(model,  meta_opt, lossfn, meta_batch):
    outer_loss = torch.tensor(0., device=device)
    outer_acc  = 0

    for task in tq.tqdm(meta_batch):
        support_loader = task['support']
        query_loader   = task['query']

        token, mask, segment, y   = next(iter(support_loader))
        preds      = nli_model(token, mask, segment)
        inner_loss = lossfn(preds, y)
        inner_acc  = get_accuracy(preds, y)

        #print("Task Loss : ", inner_loss.item()/len(support_loader), " Accuracy : ", inner_acc)
        custom_params = gradient_update_parameters(nli_model,  inner_loss,  step_size=step_alpha)

        token, mask, segment, y  = next(iter(query_loader))
        preds       = nli_model(token, mask, segment,  params=custom_params)
        outer_loss += lossfn(preds, y)
        outer_acc  += get_accuracy(preds, y)

        #del inner_loss, inner_acc, custom_params, token, mask, segment, y

    outer_loss.backward()
    print("Meta Training  Loss : ", round(outer_loss.item()/ len(meta_batch) , 5) , " Accuracy : ", round(outer_acc/ len(meta_batch),5) )
    meta_opt.step()
    del outer_loss, outer_acc, meta_batch, inner_loss, inner_acc, custom_params, token, mask, segment, y
    torch.cuda.empty_cache()

def meta_test(model, lossfn, meta_batch):

    outer_loss = 0
    outer_acc  = 0

    for task in tq.tqdm(meta_batch):
        support_loader = task['support']
        query_loader   = task['query']

        token, mask, segment, y   = next(iter(support_loader))
        preds      = nli_model(token, mask, segment)
        inner_loss = lossfn(preds, y)
        inner_acc  = get_accuracy(preds, y)

        custom_params = gradient_update_parameters(nli_model,  inner_loss,  step_size=step_alpha)

        token, mask, segment, y  = next(iter(query_loader))
        preds       = nli_model(token, mask, segment,  params=custom_params)
        outer_loss += lossfn(preds, y).item()
        outer_acc  += get_accuracy(preds, y)

        del inner_loss, inner_acc, custom_params, token, mask, segment, y

    print("Meta Testing  Loss : ", round(outer_loss/ len(meta_batch), 5) , "  Accuracy : ", round(outer_acc/ len(meta_batch), 5) )
    del outer_loss, outer_acc, meta_batch
    torch.cuda.empty_cache()


def train(model, meta_opt, lossfn, dataset, num_epochs = 1):
    for epoch in range(num_epochs):
        meta_opt.zero_grad()
        print(' Epoch  :  ', epoch , sep = ' ')
        meta_train_batch = get_batch(dataset, 200, 100)
        meta_test_batch  = get_batch(dataset, 50, 100)

        meta_train(model, meta_opt, lossfn, meta_train_batch)
        meta_test(model, lossfn, meta_test_batch)
    torch.cuda.empty_cache()

In [16]:
def predict(model, pair):
    model.eval()
    with torch.no_grad():
        tokens, segments, masks = process_pair(pair[0], pair[1], MAX_LENGTH)
        preds = model(tokens.unsqueeze(0), masks.unsqueeze(0), segments.unsqueeze(0))
    return preds.data.topk(1)[1][0].item()

### EXECUTIONS

In [17]:
import pandas as pd

In [19]:
train_langs = ['bengali', 'telugu', 'gujarati']
test_langs = ['hindi']
df_train, df_test = read_file(location, train_langs, test_langs)

META TRAINING

In [20]:
BERT_hidden_size = 768
num_classes = 2
MAX_LENGTH = 56
step_alpha = 1e-5
step_beta = 1e-6
nli_model = BERT_for_NLI(BERT_Model, 0.1).to(device)
meta_optimizer = torch.optim.Adam(nli_model.parameters(), lr=step_beta)
lossfn = nn.CrossEntropyLoss()
nli_model.load_state_dict(torch.load(location + 'bertbestinit.pt'))

<All keys matched successfully>

In [21]:
train(nli_model, meta_optimizer, lossfn , df_train, num_epochs = 150)

 Epoch  :   0



Meta Training  Loss :  0.76209  Accuracy :  0.5



Meta Testing  Loss :  0.74256   Accuracy :  0.5
 Epoch  :   1



Meta Training  Loss :  0.75194  Accuracy :  0.50333



Meta Testing  Loss :  0.71378   Accuracy :  0.5
 Epoch  :   2


...


Meta Testing  Loss :  0.71936   Accuracy :  0.5
 Epoch  :   148



Meta Training  Loss :  0.7148  Accuracy :  0.49667



Meta Testing  Loss :  0.75929   Accuracy :  0.50333
 Epoch  :   149



Meta Training  Loss :  0.74563  Accuracy :  0.49333



Meta Testing  Loss :  0.74349   Accuracy :  0.5


In [22]:
train_instance, test_instance = train_test_split(df_test[test_langs[0]] , test_size = 0.33, shuffle= True)

Few Shot Testing

In [23]:
def basic_train(model, dataset, num_epochs = 1):
    opt = torch.optim.Adam(nli_model.parameters(), lr=0.0001)
    lossfn = nn.CrossEntropyLoss()

    dataset_processed = process_dataframe(dataset, MAX_LENGTH)
    dataloader = torch.utils.data.DataLoader(dataset_processed, batch_size = 64)

    nli_model.train()
    for epoch in range(num_epochs):
        lossv = 0

        for token, mask, segment, y  in tq.tqdm(dataloader):
            opt.zero_grad()        
            preds      = nli_model(token, mask, segment)
            loss       = lossfn(preds, y)
            lossv += loss.item()
            loss.backward()
            opt.step()
        print("Loss : ", lossv/len(dataset))        

In [27]:
basic_train(nli_model, train_instance.sample(100), num_epochs = 50 )


Loss :  0.014981001019477844



Loss :  0.014559357166290284



Loss :  0.015005568265914917



Loss :  0.013978869915008544



Loss :  0.013685379028320312


...


Loss :  0.007707729637622833



Loss :  0.0077773848176002506



Loss :  0.007811700105667114



Loss :  0.007831451892852783



Loss :  0.007561057209968567


In [26]:
pred_count = 0
nli_model.eval()
for i in tq.tqdm(test_instance.index):
    pred = predict(nli_model,  [test_instance['Context'][i], test_instance['Hypothesis'][i]])
    if(pred == int(test_instance['Entailment_Label'][i] =='entailed')):
        pred_count = pred_count+1
print(pred_count, '  Accuracy :',pred_count/len(test_instance) )


477   Accuracy : 0.4818181818181818


In [28]:
pred_count = 0
nli_model.eval()
for i in tq.tqdm(test_instance.index):
    pred = predict(nli_model,  [test_instance['Context'][i], test_instance['Hypothesis'][i]])
    if(pred == int(test_instance['Entailment_Label'][i] =='entailed')):
        pred_count = pred_count+1
print(pred_count, '  Accuracy :',pred_count/len(test_instance) )


486   Accuracy : 0.4909090909090909


Few Shot Testing

### GPU Memory Tracking

In [ ]:
!nvidia-smi

In [ ]:
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

In [ ]:
df = pd.read_csv( '/content/XNLI-1.0/xnli.dev.tsv', sep = '\t')
df  = df[ df.language=='hi']
df  = df[['sentence1', 'sentence2', 'gold_label']]
df = df.reset_index()
df = df.rename(columns={'sentence1':'Context',	'sentence2' :'Hypothesis', 'gold_label' : 'Entailment_Label'})
df.to_csv(location + 'XNLI.csv')